In [1]:
!pip install imgaug
!pip install Cython
!pip install pycocotools
!pip install kaggle

In [0]:
!mkdir Hackathon

In [0]:
import os
os.chdir('Hackathon')


In [4]:
!git clone https://github.com/matterport/Mask_RCNN

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.81 MiB | 11.39 MiB/s, done.
Resolving deltas: 100% (571/571), done.


In [0]:
import os 
os.chdir('Mask_RCNN/samples')


In [6]:
"""
Mask R-CNN
Train on the toy Balloon dataset and implement color splash effect.
Copyright (c) 2018 Matterport, Inc.
Licensed under the MIT License (see LICENSE for details)
Written by Waleed Abdulla
------------------------------------------------------------
Usage: import the module (see Jupyter notebooks for examples), or run from
       the command line as such:
    # Train a new model starting from pre-trained COCO weights
    python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=coco
    # Resume training a model that you had trained earlier
    python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=last
    # Train a new model starting from ImageNet weights
    python3 balloon.py train --dataset=/path/to/balloon/dataset --weights=imagenet
    # Apply color splash to an image
    python3 balloon.py splash --weights=/path/to/weights/file.h5 --image=<URL or path to file>
    # Apply color splash to video using the last weights you trained
    python3 balloon.py splash --weights=last --video=<URL or path to file>
"""

import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

#from samples.balloon import balloon

%matplotlib inline
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


In [0]:
class TrafficLightConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
  
      """
    # Give the configuration a recognizable name
    NAME = "TrafficLight"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + TrafficLight

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5


############################################################
#  Dataset
############################################################

class TrafficLightDataset(utils.Dataset):

    def load_trafficlight(self, dataset_dir, subset):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("shapes", 1, "trafficlight")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        for a in annotations:
          annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']] 

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("Polygons")
            print(polygons)
            print("Filename")
            print(a['filename'])
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "balloon",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)

        
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "balloon":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "balloon":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = BalloonDataset()
    dataset_train.load_balloon(args.dataset, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = BalloonDataset()
    dataset_val.load_balloon(args.dataset, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads')



In [0]:
!mkdir dataset
os.chdir('dataset')

In [9]:
print(os.getcwd())
!mkdir train
os.chdir('train')


/content/Hackathon/Mask_RCNN/samples/dataset


In [13]:
# Upload images 
from google.colab import files
uploadedFile2 = files.upload()

Saving additional.zip to additional.zip


In [15]:
from google.colab import files
uploadedFile5 = files.upload()

Saving via_region_data.json to via_region_data.json


In [16]:
!unzip 'additional.zip'

Archive:  additional.zip
   creating: additional/
   creating: additional/2015-10-05-10-52-01_bag/
  inflating: additional/2015-10-05-10-52-01_bag/24594.png  
  inflating: additional/2015-10-05-10-52-01_bag/24664.png  
  inflating: additional/2015-10-05-10-52-01_bag/24734.png  
   creating: additional/2015-10-05-10-55-33_bag/
  inflating: additional/2015-10-05-10-55-33_bag/56988.png  
  inflating: additional/2015-10-05-10-55-33_bag/57058.png  
  inflating: additional/2015-10-05-10-55-33_bag/57128.png  
  inflating: additional/2015-10-05-10-55-33_bag/57196.png  
  inflating: additional/2015-10-05-10-55-33_bag/57266.png  
  inflating: additional/2015-10-05-10-55-33_bag/57336.png  
  inflating: additional/2015-10-05-10-55-33_bag/57406.png  
  inflating: additional/2015-10-05-10-55-33_bag/58576.png  
  inflating: additional/2015-10-05-10-55-33_bag/58646.png  
  inflating: additional/2015-10-05-10-55-33_bag/59388.png  
  inflating: additional/2015-10-05-10-55-33_bag/59526.png  
  inflating:

In [0]:
config = TrafficLightConfig()
TRAFFICLIGHT_DIR = os.path.join(ROOT_DIR, "samples/dataset")

In [24]:

dataset = TrafficLightDataset()
dataset.load_trafficlight(TRAFFICLIGHT_DIR, "train")


/content/Hackathon/Mask_RCNN
./rgb/additional/2015-10-05-10-52-01_bag/24594.png
./rgb/additional/2015-10-05-10-52-01_bag/24664.png
./rgb/additional/2015-10-05-10-52-01_bag/24734.png
./rgb/additional/2015-10-05-10-55-33_bag/56988.png
[278.7432988131]
[278.7432988131, 527.1012443072]
[278.7432988131, 527.1012443072]
[278.7432988131, 527.1012443072, 518.7379907959]
[278.7432988131, 527.1012443072, 518.7379907959, 296.3838188664]
[278.7432988131, 527.1012443072, 518.7379907959, 296.3838188664]
./rgb/additional/2015-10-05-10-55-33_bag/57058.png
[287.1471490007]
[287.1471490007, 140.2452731942]
[287.1471490007, 140.2452731942]
[287.1471490007, 140.2452731942, 125.7231604824]
[287.1471490007, 140.2452731942, 125.7231604824, 312.4555852231]
[287.1471490007, 140.2452731942, 125.7231604824, 312.4555852231]
[287.1471490007, 140.2452731942, 125.7231604824, 312.4555852231, 257.8928460996]
[287.1471490007, 140.2452731942, 125.7231604824, 312.4555852231, 257.8928460996, 399.345953546]
[287.1471490007